This transforms a file containing a bunch of PGNs to a collection of len 1024 batches that begin with a delimiter token ";".
Note that stockfish data has a couple of [player_one][player_two], two newlines, and then the PGN transcript.

In [ ]:
import pandas as pd
from tqdm import tqdm

# Create an input dataset containing only the transcript column and stripping the PGN header

def transform_text(text):
    if '\n\n' not in text:
        return text[:1023]
    new_text = text.split('\n\n')[1].strip()
    return new_text[:1023]

# input_file = 'lichess_6gb.csv'
input_file = 'stockfish_dataset.csv'
output_filename = input_file.replace('.csv', '_blocks.csv')


In [ ]:
df = pd.read_csv(input_file, usecols=['transcript'])
df['transcript'] = df['transcript'].apply(transform_text)

# df.to_csv('input_dataset.csv', index=False)

In [ ]:
import pandas as pd

# df = pd.read_csv('input_dataset.csv', usecols=['transcript'])
df['length'] = df['transcript'].apply(len)

df.sort_values(by='length', inplace=True)
# df.to_csv('input_dataset.csv', index=False)

In [ ]:
# import pandas as pd

# # Read the CSV file
# df = pd.read_csv('input_dataset.csv')

# # Randomly sample 10000 rows
# sampled_df = df.sample(n=10000)

# # Sort the sampled DataFrame by the length of the 'transcript' column
# sampled_df.sort_values(by='transcript', key=lambda x: x.str.len(), ascending=True, inplace=True)

# # Save the sampled and sorted DataFrame to a new CSV file
# sampled_df.to_csv('test_input_dataset.csv', index=False)


In [ ]:
import pandas as pd
from collections import deque
from tqdm import tqdm  # Import tqdm
import random

# Read the CSV file
# df = pd.read_csv('input_dataset.csv')

# Prepare the new dataset for blocks
blocks = []
remaining_games = deque(df['transcript'].tolist())  # Use deque for efficient pops from the left

del df

original_length = len(remaining_games)  # Store the original length

# Block size limit
block_size = 1024

# Initialize the progress bar
with tqdm(total=original_length, desc="Processing") as pbar:
    while remaining_games:
        block = ';'
        # Select the next game
        next_game = remaining_games.pop()
        block += next_game
        while len(block) < block_size and remaining_games:
            # if len(df) > 21:
            #     random_idx = random.randint(0, 20)
            #     next_game = remaining_games[random_idx]
            #     remaining_games[random_idx] = ""
            # else:
            next_game = remaining_games.popleft()
            block += ';' + next_game
            if len(block) > block_size:
                # If the game makes the block too long, re-add it to the dataset
                if len(remaining_games) > 100:
                    remaining_games.insert(99, next_game)
                else:
                    break
                break

        if len(block) >= block_size:
            # Add the block to the blocks list
            blocks.append(block[:block_size])

        # Update the progress bar
        pbar.update(original_length - len(remaining_games) - pbar.n)

# Create a new DataFrame for the blocks
df = pd.DataFrame(blocks, columns=['transcript'])

# Save the blocks to a new CSV file
# df.to_csv(output_filename, index=False)


In [ ]:
import pandas as pd

# temp_output_filename = "full2_dataset.csv"
# output_filename = temp_output_filename

# df = pd.read_csv(output_filename)
df = df.sample(frac=1).reset_index(drop=True)

# Save the shuffled DataFrame to the same CSV file
df.to_csv(output_filename, index=False)

df['length'] = df['transcript'].apply(len)
print(df['length'].describe())